In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.append("lib")

import lib

import torch
torch.set_default_dtype(torch.float64)

from functorch import vmap
from torch.utils.data import DataLoader
import tqdm

/home/peiyi/mambaforge3/envs/rom/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda")

In [3]:
coords_x, coords_y, _ = lib.utils.read_coords("2d_burger_data/time_step_0.csv")
coords = torch.stack((coords_x, coords_y), dim=1)

In [4]:
num_steps = 501
num_nodes = coords_x.shape[0]

datas = lib.utils.read_data(num_steps, num_nodes, '2d_burger_data/time_step_', 'vel_0')
datas.shape

torch.Size([501, 14641])

In [5]:
N = datas.shape[1]
n = 20
mu = int(np.ceil(N/40))
neighbours = lib.utils.topk_neighbours(coords, mu)

In [7]:
batch_size = 30
train_data = DataLoader(datas, batch_size=batch_size, shuffle=True)

In [9]:
ed = lib.nrbs.EncoderDecoder(N=N, n=n, mu=mu, neighbours=neighbours, device=device)

In [10]:
ed.train(train_data_loader=train_data, epochs=1)

100%|██████████| 17/17 [00:07<00:00,  2.43it/s]

Itr 0, loss = 14570.020835845391


In [11]:
ed.nrbs(datas.to(device))

tensor([[ 0.0001,  0.0051,  0.0167,  ...,  0.0016, -0.0062,  0.0086],
        [ 0.0001,  0.0051,  0.0167,  ...,  0.0017, -0.0062,  0.0086],
        [ 0.0001,  0.0051,  0.0166,  ...,  0.0017, -0.0062,  0.0086],
        ...,
        [ 0.0231,  0.0306,  0.0290,  ..., -0.0005, -0.0072, -0.0088],
        [ 0.0232,  0.0307,  0.0291,  ..., -0.0005, -0.0072, -0.0088],
        [ 0.0232,  0.0307,  0.0291,  ..., -0.0005, -0.0072, -0.0088]],
       device='cuda:0', grad_fn=<MmBackward0>)